# Concat issues

MyHDL handles booleans as concat arg, but does not warn early nor resolve to valid VHDL.

In [1]:
import sys
sys.path.insert(0, "../../")

In [2]:
import myhdl

Signal = myhdl.Signal
modbv = myhdl.modbv

@myhdl.block
def unit(clk):
    a = [ Signal(bool() ) for _ in range(4) ]
    b = Signal(modbv()[4:])
    
    c = Signal(modbv(0)[4:])

    
    @myhdl.always_comb
    def worker():
        b.next = myhdl.concat(c >= 0x80, c[3:])
        
    @myhdl.always(clk.posedge)
    def counter():
        c.next = c + 1
        
    return worker, counter

clk = Signal(bool())
inst = unit(clk)
inst.convert("VHDL")

/home/testing/.local/lib/python3.8/site-packages/myhdl/conversion/_toVHDL.py:467: ToVHDLWarning: Signal is driven but not read: b
  warnings.warn("%s: %s" % (_error.UnreadSignal, s._name),


In [3]:
!cat -n unit.vhd

     1	-- File: unit.vhd
     2	-- Generated by MyHDL 0.11
     3	-- Date: Tue Sep 21 13:21:57 2021
     4	
     5	
     6	library IEEE;
     7	use IEEE.std_logic_1164.all;
     8	use IEEE.numeric_std.all;
     9	use std.textio.all;
    10	
    11	use work.pck_myhdl_011.all;
    12	
    13	entity unit is
    14	    port (
    15	        clk: in std_logic
    16	    );
    17	end entity unit;
    18	
    19	
    20	architecture MyHDL of unit is
    21	
    22	
    23	
    24	signal b: unsigned(3 downto 0);
    25	signal c: unsigned(3 downto 0);
    26	
    27	begin
    28	
    29	
    30	
    31	
    32	
    33	b <= unsigned'((c >= 128) & c(3-1 downto 0));
    34	
    35	UNIT_COUNTER: process (clk) is
    36	begin
    37	    if rising_edge(clk) then
    38	        c <= (c + 1);
    39	    end if;
    40	end process UNIT_COUNTER;
    41	
    42	end architecture MyHDL;


In [4]:
! ghdl -i unit.vhd pck_myhdl_011.vhd

In [5]:
! ghdl -m unit

unit.vhd:33:27:error: no function declarations for operator "&"


# MyIRL emulation

In [6]:
from myirl.emulation.myhdl import *
from myirl.test.common_test import run_ghdl

from myirl.wire import SLV

@block
def unit1(clk):
    a = [ Signal(bool() ) for _ in range(4) ]
    b = Signal(modbv()[4:])
    
    c = Signal(modbv(0)[4:])

    
    @always_comb
    def worker():
        b.next = concat(c >= 0x8, c == 0, c[2:])
        
    @always(clk.posedge)
    def counter():
        c.next = c + 1
        
    return instances()

In [7]:
def convert():
    clk = Signal(bool())
    inst = unit1(clk)
    files = inst.elab(targets.VHDL)
    run_ghdl(files, inst, std="93c", debug = True)
    
convert()

../../myirl/kernel/components.py:1239: UserWarning: Non-ClkSignal 'clk' edge event reference.
You might want to use ClkSignal instead.
  base.warnings.warn("Non-ClkSignal '%s'" % obj.identifier + \


Creating process 'unit1/counter' with sensitivity (clk'rising,)
 Insert unit unit1_s1 
 DEBUG: Writing 'unit1' to file /tmp/unit1.vhdl 
Finished _elab in 0.0013 secs
==== COSIM stdout ====

==== COSIM stderr ====

==== COSIM stdout ====
analyze ../../myirl/targets/../test/vhdl/txt_util.vhdl
analyze ../../myirl/targets/libmyirl.vhdl
analyze /tmp/unit1.vhdl
elaborate unit1

==== COSIM stderr ====

==== COSIM stdout ====
../../src/ieee/v93/numeric_std-body.vhdl:1461:7:@0ms:(assertion warning): NUMERIC_STD.">=": metavalue detected, returning FALSE
../../myirl/targets/libmyirl.vhdl:45:16:@0ms:(assertion error): Warning: boolean cast to unsigned
../../src/ieee/v93/numeric_std-body.vhdl:1613:7:@0ms:(assertion warning): NUMERIC_STD."=": metavalue detected, returning FALSE
../../myirl/targets/libmyirl.vhdl:45:16:@0ms:(assertion error): Warning: boolean cast to unsigned

==== COSIM stderr ====



In [8]:
! cat /tmp/unit1.vhdl 

-- File generated from /usr/local/lib/python3.8/runpy.py
-- (c) 2016-2021 section5.ch
-- Modifications may be lost

library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;

library work;

use work.txt_util.all;
use work.myirl_conversion.all;

entity unit1 is
    port (
        clk : in std_ulogic
    );
end entity unit1;

architecture MyIRL of unit1 is
    -- Local type declarations
    -- Signal declarations
    signal b : unsigned(3 downto 0);
    signal c : unsigned(3 downto 0);
begin
    
worker:
    process(c)
    begin
        b <= (unsigned'(from_bool((c >= x"8"))) & unsigned'(from_bool((c = x"0"))) & c(2-1 downto 0));
    end process;

    
counter:
    process(clk)
    begin
        if rising_edge(clk) then
            c <= resize(((resize((c), 5) + "00001")), 4);
        end if;
    end process;
end architecture MyIRL;

